In [4]:
# EDA
import pandas as pd
import pingouin as pg
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.figure_factory as ff

# Machine Learning
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

# Otimização de Hiperparâmetros
import optuna

c:\Users\danil\.virtualenvs\classificador-clientes-IA-CUWjydxM\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Carregar dataset
df_segmentos = pd.read_csv('./datasets/dataset_segmentos_clientes.csv')

### EDA

In [6]:
df_segmentos.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao,segmento_de_cliente
0,Comércio,713109.95,12,Rio de Janeiro,6,1,Bronze
1,Comércio,790714.38,9,São Paulo,15,0,Bronze
2,Comércio,1197239.33,17,São Paulo,4,9,Silver
3,Indústria,449185.78,15,São Paulo,6,0,Starter
4,Agronegócio,1006373.16,15,São Paulo,15,8,Silver
5,Serviços,1629562.41,16,Rio de Janeiro,11,4,Silver
6,Serviços,771179.95,13,Vitória,0,1,Starter
7,Serviços,707837.61,16,São Paulo,10,6,Silver
8,Comércio,888983.66,17,Belo Horizonte,10,1,Bronze
9,Indústria,1098512.64,13,Rio de Janeiro,9,3,Bronze


In [7]:
df_segmentos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   atividade_economica     500 non-null    object 
 1   faturamento_mensal      500 non-null    float64
 2   numero_de_funcionarios  500 non-null    int64  
 3   localizacao             500 non-null    object 
 4   idade                   500 non-null    int64  
 5   inovacao                500 non-null    int64  
 6   segmento_de_cliente     500 non-null    object 
dtypes: float64(1), int64(3), object(3)
memory usage: 27.5+ KB


In [8]:
# Valores possíveis - Variaveis categóricas
df_segmentos['atividade_economica'].unique()

array(['Comércio', 'Indústria', 'Agronegócio', 'Serviços'], dtype=object)

In [9]:
# Valores possíveis - Variaveis categóricas
df_segmentos['localizacao'].unique()

array(['Rio de Janeiro', 'São Paulo', 'Vitória', 'Belo Horizonte'],
      dtype=object)

In [10]:
# Valores possíveis - Variaveis categóricas
df_segmentos['inovacao'].unique()

array([1, 0, 9, 8, 4, 6, 3, 7, 5, 2])

In [11]:
# Valores possíveis - Variaveis categóricas
df_segmentos['segmento_de_cliente'].unique()

array(['Bronze', 'Silver', 'Starter', 'Gold'], dtype=object)

In [12]:
# Distribuição da variável segmento_de_cliente (target)
contagem_target = df_segmentos.value_counts('segmento_de_cliente')
contagem_target

segmento_de_cliente
Silver     260
Bronze     202
Starter     22
Gold        16
Name: count, dtype: int64

In [13]:
lista_segmentos = ['Starter', 'Bronze', 'Silver', 'Gold']

In [14]:
# Distribuição da variável target - Contagem
percentual_target = contagem_target / len(df_segmentos) * 100
px.bar(percentual_target, color=percentual_target.index, category_orders={'segmento_de_cliente': lista_segmentos})

In [15]:
# Distribuição da variável localizacao
percentual_localizacao = df_segmentos.value_counts('localizacao') / len(df_segmentos) * 100
px.bar(percentual_localizacao, color=percentual_localizacao.index)

In [16]:
# Distribuição da variável atividade_economica
percentual_atividade = df_segmentos.value_counts('atividade_economica') / len(df_segmentos) * 100
px.bar(percentual_atividade, color=percentual_atividade.index)

In [17]:
# Distribuição da variável inovacao
percentual_inovacao = df_segmentos.value_counts('inovacao') / len(df_segmentos) * 100
px.bar(percentual_inovacao, color=percentual_inovacao.index)

In [18]:
# Tabela de contigência entre localização e target
crosstab_localizacao = pd.crosstab(df_segmentos['localizacao'], df_segmentos['segmento_de_cliente'], margins=True)[lista_segmentos].reset_index()
tabela_localizacao = ff.create_table(crosstab_localizacao)
tabela_localizacao.show()

In [19]:
# Tabela de contigência entre atividade e target
crosstab_atividade = pd.crosstab(df_segmentos['atividade_economica'], df_segmentos['segmento_de_cliente'], margins=True)[lista_segmentos].reset_index()
tabela_atividade= ff.create_table(crosstab_atividade)
tabela_atividade.show()

In [20]:
# Tabela de contigência entre inovação e target
crosstab_inovacao = pd.crosstab(df_segmentos['inovacao'], df_segmentos['segmento_de_cliente'], margins=True)[lista_segmentos].reset_index()
tabela_inovacao = ff.create_table(crosstab_inovacao)
tabela_inovacao.show()

In [21]:
# Distribuição da idade da empresa e target
px.histogram(df_segmentos['idade'])

In [22]:
# Distribuição da Faturamento Mensal da empresa e target
px.histogram(df_segmentos['faturamento_mensal'])

In [23]:
# boxPlot entre idade da empresa e segmento
px.box(df_segmentos, x='segmento_de_cliente', y='idade', color='segmento_de_cliente', category_orders={'segmento_de_cliente': lista_segmentos})

In [24]:
# boxPlot entre Faturamento Mensal da empresa e segmento
px.box(df_segmentos, x='segmento_de_cliente', y='faturamento_mensal', color='segmento_de_cliente', category_orders={'segmento_de_cliente': lista_segmentos})

In [43]:
# Teste de Qui-Quadrado de Pearson
valor_esperado, valor_observado, estatistica = pg.chi2_independence(df_segmentos, 'inovacao', 'segmento_de_cliente')

c:\Users\danil\.virtualenvs\classificador-clientes-IA-CUWjydxM\Lib\site-packages\pingouin\contingency.py:151: UserWarning:

Low count on observed frequencies.

c:\Users\danil\.virtualenvs\classificador-clientes-IA-CUWjydxM\Lib\site-packages\pingouin\contingency.py:151: UserWarning:

Low count on expected frequencies.

c:\Users\danil\.virtualenvs\classificador-clientes-IA-CUWjydxM\Lib\site-packages\scipy\stats\_stats_py.py:7409: RuntimeWarning:

divide by zero encountered in power

c:\Users\danil\.virtualenvs\classificador-clientes-IA-CUWjydxM\Lib\site-packages\scipy\stats\_stats_py.py:7409: RuntimeWarning:

invalid value encountered in multiply

c:\Users\danil\.virtualenvs\classificador-clientes-IA-CUWjydxM\Lib\site-packages\scipy\stats\_stats_py.py:7406: RuntimeWarning:

divide by zero encountered in divide



In [44]:
valor_esperado

segmento_de_cliente,Bronze,Gold,Silver,Starter
inovacao,,,,
0,21.008,1.664,27.04,2.288
1,23.028,1.824,29.64,2.508
2,22.624,1.792,29.12,2.464
3,19.392,1.536,24.96,2.112
4,17.372,1.376,22.36,1.892
5,17.372,1.376,22.36,1.892
6,21.816,1.728,28.08,2.376
7,21.412,1.696,27.56,2.332
8,18.988,1.504,24.44,2.068


In [45]:
valor_observado

segmento_de_cliente,Bronze,Gold,Silver,Starter
inovacao,,,,
0,36,0,10,6
1,44,0,5,8
2,32,0,20,4
3,22,0,25,1
4,12,0,30,1
5,14,3,25,1
6,15,0,38,1
7,12,5,36,0
8,9,4,34,0


In [46]:
estatistica.round(5)

,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.00000,164.29399,27.0,0.0,0.33095,0.99850
1,cressie-read,0.66667,165.49946,27.0,0.0,0.33216,0.99861
2,log-likelihood,0.00000,181.48878,27.0,0.0,0.34784,0.99951
3,freeman-tukey,-0.50000,NaN,27.0,NaN,NaN,NaN
4,mod-log-likelihood,-1.00000,inf,27.0,0.0,inf,NaN
5,neyman,-2.00000,NaN,27.0,NaN,NaN,NaN


As variáveis Localização e Segmento de Cliente são independentes. Qui-Quadrado = (p-value = 0.81714)                                                                         
As variáveis Atividade Econômica e Segmento de Cliente são independentes. Qui-Quadrado = (p-value = 0.35292)                                              
As variáveis Inovação e Segmento de Cliente não são independentes. Qui-Quadrado (p-value = 0.0)

### Treinamento do Modelo

In [49]:
X = df_segmentos.drop(columns=['segmento_de_cliente'])
y = df_segmentos['segmento_de_cliente']

In [54]:
# Pipeline
categorical_features = ['localizacao', 'atividade_economica']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)]
)

dt_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier())
])

### Validação Cruzada

In [56]:
# Treinar o modelo com StratifiedKFold, já que os dados estão desbalanceados
cv_folds = StratifiedKFold(n_splits=3, shuffle=True, random_state=51)
metrics_result = cross_validate(dt_model, X, y, cv=cv_folds, scoring=['accuracy'], return_estimator=True)

In [ ]:
# Mostrar retorno do Cross Validation
metrics_result

{'fit_time': array([0.00472188, 0.00438166, 0.00363922]),
 'score_time': array([0.00242496, 0.00239682, 0.00265503]),
 'estimator': [Pipeline(steps=[('preprocessor',
                   ColumnTransformer(transformers=[('cat',
                                                    Pipeline(steps=[('imputer',
                                                                     SimpleImputer(strategy='most_frequent')),
                                                                    ('onehot',
                                                                     OneHotEncoder(handle_unknown='ignore'))]),
                                                    ['localizacao',
                                                     'atividade_economica'])])),
                  ('classifier', DecisionTreeClassifier())]),
  Pipeline(steps=[('preprocessor',
                   ColumnTransformer(transformers=[('cat',
                                                    Pipeline(steps=[('imputer',
        

In [58]:
# Média da acurácia, considerando os 3 splits
metrics_result['test_accuracy'].mean()

np.float64(0.4739917754851743)